In [3]:
import os
import subprocess
from cartesia import Cartesia
import time
from IPython.display import Audio

client = Cartesia(api_key="")

TypeError: a bytes-like object is required, not 'generator'

In [29]:
st = time.time()
data = client.tts.bytes(
    model_id="sonic-2",
    transcript="Let's create a clip!. But this is just a test",
    voice={
    "mode": "id",
    "id": "694f9389-aac1-45b6-b726-9d9369183238"
  },  # Barbershop Man
    # You can find the supported `output_format`s at https://docs.cartesia.ai/api-reference/tts/bytes
    output_format={
        "container": "wav",
        "encoding": "pcm_f32le",
        "sample_rate": 24000,
    },
)
# audio_bytes = b"".join(data)  # generator에서 모든 chunk를 합침
# with open("sonic-2.wav", "wb") as f:
#     f.write(audio_bytes)

with open("sonic-2.wav", "wb") as f:
    for chunk in data:   # generator이므로 반복문으로 돌려야 함
        f.write(chunk)
print(time.time() - st)

# Play the file
# subprocess.run(["ffplay", "-autoexit", "-nodisp", "sonic-2.wav"])
display(Audio("sonic-2.wav"))

1.7676777839660645


In [30]:
from resemble import Resemble
import time

Resemble.api_key('')

# Get your default Resemble project.
project_uuid = Resemble.v2.projects.all(1, 10)['items'][0]['uuid']

# Get your Voice uuid. In this example, we'll obtain the first.
voice_uuid = Resemble.v2.voices.all(1, 10)['items'][0]['uuid']

# Let's create a clip!
st = time.time()
body = "Let's create a clip!. But this is just a test"
response = Resemble.v2.clips.create_sync(project_uuid,
                                         voice_uuid,
                                         body,
                                         title=None,
                                         sample_rate=22050,
                                         output_format=None,
                                         precision=None,
                                         include_timestamps=False,
                                         is_archived=None,
                                         raw=True)
print(time.time() - st)

2.3257384300231934


In [32]:
import requests
st = time.time()
# Create speech (POST /v1/text-to-speech/:voice_id)
response = requests.post(
  "https://api.elevenlabs.io/v1/text-to-speech/JBFqnCBsd6RMkjVDRZzb?output_format=mp3_22050_32",
  headers={
    "xi-api-key": ""
  },
  json={
    "text": "The first move is what sets everything in motion.",
    "model_id": "eleven_flash_v2_5"
  },
)

# 저장
with open("output.mp3", "wb") as f:
    f.write(response.content)
print(time.time() - st)

print("Saved as output.mp3")

0.4506716728210449
Saved as output.mp3


In [ ]:
from elevenlabs import ElevenLabs
import time

from pydub import AudioSegment

# 오디오 파일 로드 (wav, mp3 등 지원)
audio = AudioSegment.from_file("eo.m4a")

# 앞 5초(5000ms)만 잘라내기
first_5s = audio[:10000]

# 파일로 저장
first_5s.export("output.wav", format="wav")

client = ElevenLabs(
    api_key="",
)
file = open("./output.wav", "rb")

st = time.time()
response = client.speech_to_text.convert(
    model_id="scribe_v1_experimental",
    file=file,
    language_code="ko",
    tag_audio_events=False
)
print(time.time() - st)
print(response.text)

In [ ]:
import os
import json
base_dir = "../../../../dataset/라벨링데이터_230316"
json_files = []

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            json_files.append(os.path.join(root, file))

print(len(json_files))  # 찾은 json 파일 개수
print(json_files[:10])  # 앞 10개만 확인

1926
['../../../../dataset/라벨링데이터_230316/D01/J01/S002560/S002560.json', '../../../../dataset/라벨링데이터_230316/D01/J01/S002558/S002558.json', '../../../../dataset/라벨링데이터_230316/D01/J01/S002551/S002551.json', '../../../../dataset/라벨링데이터_230316/D01/J01/S002556/S002556.json', '../../../../dataset/라벨링데이터_230316/D01/J01/S002557/S002557.json', '../../../../dataset/라벨링데이터_230316/D01/J01/S002550/S002550.json', '../../../../dataset/라벨링데이터_230316/D01/J01/S002559/S002559.json', '../../../../dataset/라벨링데이터_230316/D01/J01/S002561/S002561.json', '../../../../dataset/라벨링데이터_230316/D01/J01/S002543/S002543.json', '../../../../dataset/라벨링데이터_230316/D01/J01/S002544/S002544.json']


In [21]:
import json
from tqdm import tqdm

datas = []
for json_file in tqdm(json_files):
    with open(json_file, "r") as f:
        data = json.load(f)
    dialogs = data['dataSet']['dialogs']
    for dial in dialogs:
        audio_path = "../../../../dataset/원천데이터_230316/" + dial['audioPath']
        if not os.path.exists(audio_path):
            continue
        datas.append({
            "audio_path": audio_path,
            "text": dial['text'],
            "duration": dial['duration']
        })

print(len(datas))

100%|██████████| 1926/1926 [00:00<00:00, 3430.81it/s]

39916


In [24]:
import re
import shutil

src = "source/audio.wav"
dst = "dest/audio.wav"

# 파일 복사
for d in tqdm(datas):
    new_ap = re.sub('/', '_', d['audio_path'][len("../../../../dataset/원천데이터_230316/"):])
    shutil.copy(d['audio_path'], "../../../../dataset/audios/"+new_ap)

100%|██████████| 39916/39916 [00:17<00:00, 2235.37it/s]


In [53]:
import re, unicodedata

def strip_leading_oslash(text: str) -> str:
    # 유니코드 정규화 (형태 통일)
    t = unicodedata.normalize('NFKC', text)
    # BOM, 제로폭 문자, 공백, 따옴표류 제거
    t = re.sub(r'^[\uFEFF\u200B\u200C\u200D\u2060\s\'"“”‘’]*', '', t)
    # 맨 앞의 o/ + 뒤따르는 공백 제거
    t = re.sub(r'^o/\s*', '', t)
    return t

texts = [d['text'] for d in datas]
print(len(texts))
print(len(set(texts)))

import random

random.shuffle(datas)
new_datas = random.sample(datas, 6000)
new_datas2 = []
for d in new_datas:
    nap = re.sub('/', '_', d['audio_path'][len("../../../../dataset/원천데이터_230316/"):])
    text = d['text']
    text = re.sub(r'^n/\s*', '', text)
    text = re.sub(r'^o/\s*', '', text)
    text = strip_leading_oslash(text)

    if d['duration'] > 4 and d['duration'] < 15:
        new_datas2.append({
            "audio_path": nap,
            "text": text,
            "duration": d['duration']
        })
print(len(new_datas2))

39916
36643
3048


In [54]:
new_datas2[:5]

[{'audio_path': 'D01_J021_S007435_0006.wav',
  'text': '구매를 (하려고)/(할려고) 그러는데 구매 가능한 교재가 없습니다 이렇게 나오거든요 그 어떻게 살 수 없나요?',
  'duration': 6.349},
 {'audio_path': 'D02_J05_S007980_0017.wav',
  'text': '아 사회복지과 쪽으로 방문하시면 되시는 건가요? 아 네 알겠습니다. 확인 감사합니다.',
  'duration': 5.922},
 {'audio_path': 'D01_J01_S002560_0032.wav',
  'text': '궁금하신 부분이 과학 쪽에서 어느 쪽 과목 말씀이실까요?',
  'duration': 5.288},
 {'audio_path': 'D02_J08_S009372_0015.wav',
  'text': '아니 현장 나가시지 그렇게 막 계시면서 오늘 아침에 그렇게 얘기하실 분이 없으세요.',
  'duration': 6.494},
 {'audio_path': 'D02_J05_S007971_0009.wav',
  'text': '아 예 주무관님 혹시 여기 입소 시설 입소를 아예 하신 건가요 어르신?',
  'duration': 5.117}]

In [65]:
import re
import shutil

src = "source/audio.wav"
dst = "dest/audio.wav"

# 파일 복사
for d in tqdm(new_datas2):
    ap = '../../../../dataset/audios/' + d['audio_path']
    newap = '../../../../dataset/audios_sampled/' + d['audio_path']
    shutil.copy(ap, newap)

100%|██████████| 3048/3048 [00:01<00:00, 2226.66it/s]


In [66]:
os.path.exists(newap)

True

In [69]:
with open("output.json", "w", encoding="utf-8") as f:
    json.dump(new_datas2, f, ensure_ascii=False, indent=4)